In [1]:
# third-party
import pickle
import pandas as pd
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

# local
from results_generator import write_results
from get_data import get_participant_ids, transform_overview_on_overall
from performance import get_fr_detection_performance, get_breath_parameters_performance
from files import update_thr_comparison_file


In [2]:
from get_data import get_data_by_id_activity

acquisition_folderpath = 'Aquisicao'
id_participants = get_participant_ids(acquisition_folderpath)
data, data_raw = get_data_by_id_activity(acquisition_folderpath)

threshold_acceptability = 0.5

Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
print('Writing results...')
for id in id_participants:
    print(' ---------',id,'---------------')
    write_results(id, data[id], data_raw[id], acquisition_folderpath, show_fig=False, threshold_acceptability=threshold_acceptability)

Writing results...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
ALR
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [4]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

In [5]:
overview_all = transform_overview_on_overall(overview)
fr_performance = get_fr_detection_performance(overview_all, target=None)
bp_performance = get_breath_parameters_performance(overview_all, target=None)

for metric in ["Ratio", "Precision", "Recall"]:
    update_thr_comparison_file(threshold_acceptability, metric, fr_performance.loc[fr_performance["Sensor"]=="MAG"][metric].values[0])

for metric in ["tI", "tE", "tB"]:
    metric_performance = bp_performance.loc[bp_performance["Sensor"]=="MAG"].loc[bp_performance["Parameter"]==metric]["Rel. error (%)"].values[0].split(' $\\pm$ ')
    update_thr_comparison_file(threshold_acceptability, metric, float(metric_performance[0]))
    update_thr_comparison_file(threshold_acceptability, metric+" error SD", float(metric_performance[1]))

thr_comparison = pd.read_csv('Results/thrs_comparison.csv')
thr_comparison  

N=3357 tI for MAG
N=3275 tI for PZT
N=3351 tE for MAG
N=3261 tE for PZT
N=3263 tB for MAG
N=3185 tB for PZT


,thr,Ratio,Precision,Recall,tI,tI error SD,tE,tE error SD,tB,tB error SD
0,0.1,1.1,0.68,0.74,9.96,8.13,9.08,7.66,3.48,3.33
1,0.2,1.1,0.81,0.89,14.75,13.00,13.73,12.22,5.42,6.05
2,0.3,1.1,0.87,0.95,17.70,16.10,16.78,15.48,6.64,7.88
3,0.4,1.1,0.88,0.97,18.86,17.77,17.77,16.95,7.12,8.56
4,0.5,1.1,0.89,0.98,19.33,18.79,18.20,17.72,7.40,9.20


In [6]:
import numpy as np

print("Coefficient of variation\n")
for metric in ["Ratio", "Precision", "Recall", "tI", "tE", "tB", "tI error SD", "tE error SD", "tB error SD"]:
    print(f"{metric}: {(np.std(thr_comparison[metric]) / np.mean(thr_comparison[metric].mean()))*100:.1f}")

Coefficient of variation

Ratio: 0.0
Precision: 9.5
Recall: 9.8
tI: 21.5
tE: 22.5
tB: 23.9
tI error SD: 26.1
tE error SD: 26.3
tB error SD: 30.2


In [22]:
import plotly.graph_objects as go
from constants import CATEGORICAL_PALETTE

def plot_thr_comparison(metrics, title, ytitle, filename, yrange):
    
    fig = go.Figure()
    
    for i,metric in enumerate(metrics):
        y = 100 - thr_comparison[metric] if filename=="thr_bp_performance" else thr_comparison[metric]*100
        error_y = dict(type='data', array=thr_comparison[f"{metric} error SD"], visible=True) if filename=="thr_bp_performance" else None
        
        fig.add_trace(
            go.Scatter(
                x=thr_comparison['thr'],
                y=y,
                line=dict(color=CATEGORICAL_PALETTE[i], width=2),
                mode='lines',
                name=metric,
                # error_y=error_y
            ))


    fig.update(layout_showlegend=True, layout_height=350, layout_width=400)
    fig.update_layout(title=title,
                      margin=go.layout.Margin(
                          b=10, #bottom margin
                          r=10, #right margin
                          t=30, #top margin
                        ))
    fig.update_xaxes(title_text="Acceptability threshold", autorange="reversed")
    fig.update_yaxes(title_text=ytitle, range=yrange)
    fig.write_image(f"Results/{filename}.pdf")
    fig.show()


yrange = [pd.concat([thr_comparison[["Precision", "Recall"]] * 100, 100 - thr_comparison[["tI", "tE", "tB"]]], axis=1).min(axis=None), 100]
plot_thr_comparison(["Precision", "Recall"], "FR detection", "Performance (%)", "thr_fr_performance", yrange)
plot_thr_comparison(["tI", "tE", "tB"], "Respiratory parameters", "100 - Rel. Error (%)", "thr_bp_performance", yrange)





In [8]:
with open('Results/results.pickle', 'rb') as file:
    data = pickle.load(file)

activities_list = ['SNB', 'SGB', 'MCH', 'SQT', 'MIXB', 'STNB', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR']

In [9]:
overview = pd.DataFrame(columns=["participant_id", "activity", "FR events", "complete cycles", "BR"])


for id in id_participants:
    print('---------',id,'---------------')
    
    new_entry = {"participant_id": id}

    for activity in activities_list:
        try:
            new_entry["activity"] = activity
            new_entry["FR events"] = len(data[id][activity]["Airflow"]["peaks"]) + len(data[id][activity]["Airflow"]["valleys"])
            new_entry["complete cycles"] = len(data[id][activity]["Airflow"]["tB (s)"])
            new_entry["BR"] = data[id][activity]["Airflow"]["BR (bpm)"]

            overview.loc[len(overview)] = new_entry
        
        except KeyError as e:
            print(f"    Could not find key {e}")
            continue

overview

--------- 7OYX ---------------
--------- NO15 ---------------
--------- G8B7 ---------------
--------- EPE2 ---------------
--------- HAK8 ---------------
--------- 1BST ---------------
--------- 83J1 ---------------
--------- QMQ7 ---------------
    Could not find key 'MIXB'
--------- 9TUL ---------------
--------- FTD7 ---------------
--------- Y6O3 ---------------
--------- 2QWT ---------------
--------- F9AF ---------------
--------- P4W9 ---------------
--------- W8Z9 ---------------
--------- D4GQ ---------------


,participant_id,activity,FR events,complete cycles,BR
0,7OYX,SNB,29,13,13.026052
1,7OYX,SGB,13,5,5.818464
2,7OYX,MCH,22,10,9.789525
3,7OYX,SQT,24,11,11.909058
4,7OYX,MIXB,24,11,16.228178
...,...,...,...,...,...
234,D4GQ,UAL,22,10,10.750762
235,D4GQ,UAR,21,9,10.217597
236,D4GQ,SE,21,10,10.209290
237,D4GQ,SS,19,9,9.303928


In [10]:
# get data where activity is not UALm, UARm, SNBm
removed_middle = overview[~overview["activity"].isin(["UALm", "UARm", "SNBm"])]
print(f"mean BR: {removed_middle['BR'].mean()}")
print(f"FR events: {removed_middle['FR events'].sum()}")
print(f"complete cycles: {removed_middle['complete cycles'].sum()}")

mean BR: 14.65143676191543
FR events: 7209
complete cycles: 3363


In [11]:
overview_by_activity = pd.DataFrame(columns=["activity", "BR [bpm]", "FR events", "complete cycles"])

for activity in activities_list:
    new_entry = {"activity": activity}
    activity_data = overview[overview["activity"] == activity]
    new_entry["FR events"] = activity_data["FR events"].sum()
    new_entry["complete cycles"] = activity_data["complete cycles"].sum()
    new_entry["BR [bpm]"] = f'{np.round(np.mean(activity_data["BR"]), 1)} $\pm$ {np.round(np.std(activity_data["BR"]), 1)}'
    overview_by_activity.loc[len(overview_by_activity)] = new_entry

overview_by_activity

,activity,BR [bpm],FR events,complete cycles
0,SNB,13.4 $\pm$ 3.4,451,208
1,SGB,5.9 $\pm$ 0.1,195,80
2,MCH,15.2 $\pm$ 4.7,504,236
3,SQT,16.5 $\pm$ 4.6,541,254
4,MIXB,12.2 $\pm$ 3.0,354,161
5,STNB,13.2 $\pm$ 3.4,455,214
6,AAL,16.5 $\pm$ 3.8,542,255
7,AAR,16.5 $\pm$ 3.9,538,252
8,ALL,17.4 $\pm$ 4.2,569,267
9,ALR,17.1 $\pm$ 4.1,565,266
